In [1]:
import pandas as pd
import matplotlib.pylab as plt

from swat import *
from dlpy import Model, Sequential
from dlpy.model import Optimizer, MomentumSolver, AdamSolver, Gpu
from dlpy.layers import * 
from dlpy.speech import *
from dlpy.splitting import two_way_split
from dlpy.metrics import (accuracy_score, confusion_matrix, plot_roc, 
                          plot_precision_recall, roc_auc_score, f1_score, average_precision_score)
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

# Load the processed data into CAS

In [3]:
conn = CAS('host_name', port_number)

In [4]:
from dlpy.audio import AudioTable 

In [5]:
# a sashdat file that contain the audio data is generated using AudioTable.load_audio_files
# AudioTable.from_audio_sashdat can load the sashdat file into a CAS with a server data path
train_audio = AudioTable.from_audio_sashdat(conn, '/data_dir/data/spoken_language_identification/train_wav.sashdat')

In [6]:
train_audio.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,_id_,,1,int64,8,12,,0,0
1,_path_,,2,varchar,120,120,,0,0
2,_type_,,3,char,8,8,,0,0
3,_audio_,,4,varbinary(sound),320046,320046,,0,0


In [7]:
train_audio.numrows()

[numrows]

 73080

+ Elapsed: 0.000322s, user: 0.000143s, sys: 0.000163s, mem: 0.396mb

In [8]:
# generate MFCC features
train_feature_table = train_audio.create_audio_feature_table(label_level=-2, n_output_frames=1000, random_shuffle=True)

In [9]:
train_feature_table.fetch(to=2)

,_path_,_num_frames_,_f0_v0_,_f0_v1_,_f0_v2_,_f0_v3_,_f0_v4_,_f0_v5_,_f0_v6_,_f0_v7_,_f0_v8_,_f0_v9_,_f0_v10_,_f0_v11_,_f0_v12_,_f0_v13_,_f0_v14_,_f0_v15_,_f0_v16_,_f0_v17_,_f0_v18_,_f0_v19_,_f0_v20_,_f0_v21_,_f0_v22_,_f0_v23_,_f0_v24_,_f0_v25_,_f0_v26_,_f0_v27_,_f0_v28_,_f0_v29_,_f0_v30_,_f0_v31_,_f0_v32_,_f0_v33_,_f0_v34_,_f0_v35_,_f0_v36_,_f0_v37_,_f0_v38_,_f0_v39_,_f1_v0_,_f1_v1_,_f1_v2_,_f1_v3_,_f1_v4_,_f1_v5_,_f1_v6_,_f1_v7_,_f1_v8_,_f1_v9_,_f1_v10_,_f1_v11_,_f1_v12_,_f1_v13_,_f1_v14_,_f1_v15_,_f1_v16_,_f1_v17_,_f1_v18_,_f1_v19_,_f1_v20_,_f1_v21_,_f1_v22_,_f1_v23_,_f1_v24_,_f1_v25_,_f1_v26_,_f1_v27_,_f1_v28_,_f1_v29_,_f1_v30_,_f1_v31_,_f1_v32_,_f1_v33_,_f1_v34_,_f1_v35_,_f1_v36_,_f1_v37_,_f1_v38_,_f1_v39_,_f2_v0_,_f2_v1_,_f2_v2_,_f2_v3_,_f2_v4_,_f2_v5_,_f2_v6_,_f2_v7_,_f2_v8_,_f2_v9_,_f2_v10_,_f2_v11_,_f2_v12_,_f2_v13_,_f2_v14_,_f2_v15_,_f2_v16_,_f2_v17_,_f2_v18_,_f2_v19_,_f2_v20_,_f2_v21_,_f2_v22_,_f2_v23_,_f2_v24_,_f2_v25_,_f2_v26_,_f2_v27_,_f2_v28_,_f2_v29_,_f2_v30_,_f2_v31_,_f2_v32_,_f2_v33_,_f2_v34_,_f2_v35_,_f2_v36_,_f2_v37_,_f2_v38_,_f2_v39_,_f3_v0_,_f3_v1_,_f3_v2_,_f3_v3_,_f3_v4_,_f3_v5_,_f3_v6_,_f3_v7_,_f3_v8_,_f3_v9_,_f3_v10_,_f3_v11_,_f3_v12_,_f3_v13_,_f3_v14_,_f3_v15_,_f3_v16_,_f3_v17_,_f3_v18_,_f3_v19_,_f3_v20_,_f3_v21_,_f3_v22_,_f3_v23_,_f3_v24_,_f3_v25_,_f3_v26_,_f3_v27_,_f3_v28_,_f3_v29_,_f3_v30_,_f3_v31_,_f3_v32_,_f3_v33_,_f3_v34_,_f3_v35_,_f3_v36_,_f3_v37_,_f3_v38_,_f3_v39_,_f4_v0_,_f4_v1_,_f4_v2_,_f4_v3_,_f4_v4_,_f4_v5_,_f4_v6_,_f4_v7_,_f4_v8_,_f4_v9_,_f4_v10_,_f4_v11_,_f4_v12_,_f4_v13_,_f4_v14_,_f4_v15_,_f4_v16_,_f4_v17_,_f4_v18_,_f4_v19_,_f4_v20_,_f4_v21_,_f4_v22_,_f4_v23_,_f4_v24_,_f4_v25_,_f4_v26_,_f4_v27_,_f4_v28_,_f4_v29_,_f4_v30_,_f4_v31_,_f4_v32_,_f4_v33_,_f4_v34_,_f4_v35_,_f4_v36_,_f4_v37_,_f4_v38_,_f4_v39_,_f5_v0_,_f5_v1_,_f5_v2_,_f5_v3_,_f5_v4_,_f5_v5_,_f5_v6_,_f5_v7_,_f5_v8_,_f5_v9_,_f5_v10_,_f5_v11_,_f5_v12_,_f5_v13_,_f5_v14_,_f5_v15_,_f5_v16_,_f5_v17_,_f5_v18_,_f5_v19_,_f5_v20_,_f5_v21_,_f5_v22_,_f5_v23_,_f5_v24_,_f5_v25_,_f5_v26_,_f5_v27_,_f5_v28_,_f5_v29_,_f5_v30_,_f5_v31_,_f5_v32_,_f5_v33_,_f5_v34_,_f5_v35_,_f5_v36_,_f5_v37_,_f5_v38_,_f5_v39_,_f6_v0_,_f6_v1_,_f6_v2_,_f6_v3_,_f6_v4_,_f6_v5_,_f6_v6_,_f6_v7_,...,_f993_v32_,_f993_v33_,_f993_v34_,_f993_v35_,_f993_v36_,_f993_v37_,_f993_v38_,_f993_v39_,_f994_v0_,_f994_v1_,_f994_v2_,_f994_v3_,_f994_v4_,_f994_v5_,_f994_v6_,_f994_v7_,_f994_v8_,_f994_v9_,_f994_v10_,_f994_v11_,_f994_v12_,_f994_v13_,_f994_v14_,_f994_v15_,_f994_v16_,_f994_v17_,_f994_v18_,_f994_v19_,_f994_v20_,_f994_v21_,_f994_v22_,_f994_v23_,_f994_v24_,_f994_v25_,_f994_v26_,_f994_v27_,_f994_v28_,_f994_v29_,_f994_v30_,_f994_v31_,_f994_v32_,_f994_v33_,_f994_v34_,_f994_v35_,_f994_v36_,_f994_v37_,_f994_v38_,_f994_v39_,_f995_v0_,_f995_v1_,_f995_v2_,_f995_v3_,_f995_v4_,_f995_v5_,_f995_v6_,_f995_v7_,_f995_v8_,_f995_v9_,_f995_v10_,_f995_v11_,_f995_v12_,_f995_v13_,_f995_v14_,_f995_v15_,_f995_v16_,_f995_v17_,_f995_v18_,_f995_v19_,_f995_v20_,_f995_v21_,_f995_v22_,_f995_v23_,_f995_v24_,_f995_v25_,_f995_v26_,_f995_v27_,_f995_v28_,_f995_v29_,_f995_v30_,_f995_v31_,_f995_v32_,_f995_v33_,_f995_v34_,_f995_v35_,_f995_v36_,_f995_v37_,_f995_v38_,_f995_v39_,_f996_v0_,_f996_v1_,_f996_v2_,_f996_v3_,_f996_v4_,_f996_v5_,_f996_v6_,_f996_v7_,_f996_v8_,_f996_v9_,_f996_v10_,_f996_v11_,_f996_v12_,_f996_v13_,_f996_v14_,_f996_v15_,_f996_v16_,_f996_v17_,_f996_v18_,_f996_v19_,_f996_v20_,_f996_v21_,_f996_v22_,_f996_v23_,_f996_v24_,_f996_v25_,_f996_v26_,_f996_v27_,_f996_v28_,_f996_v29_,_f996_v30_,_f996_v31_,_f996_v32_,_f996_v33_,_f996_v34_,_f996_v35_,_f996_v36_,_f996_v37_,_f996_v38_,_f996_v39_,_f997_v0_,_f997_v1_,_f997_v2_,_f997_v3_,_f997_v4_,_f997_v5_,_f997_v6_,_f997_v7_,_f997_v8_,_f997_v9_,_f997_v10_,_f997_v11_,_f997_v12_,_f997_v13_,_f997_v14_,_f997_v15_,_f997_v16_,_f997_v17_,_f997_v18_,_f997_v19_,_f997_v20_,_f997_v21_,_f997_v22_,_f997_v23_,_f997_v24_,_f997_v25_,_f997_v26_,_f997_v27_,_f997_v28_,_f997_v29_,_f997_v30_,_f997_v31_,_f997_v32_,_f997_v33_,_f997_v34_,_f997_v35_,_f997_v36_,_f997_v37_,_f997_v38_,_f997_v39_,_f9

In [10]:
train_feature_table.label_freq

,Level,Frequency
de,1,24360
en,2,24360
es,3,24360


In [11]:
# load the test data
test_audio = AudioTable.from_audio_sashdat(conn, '/data_dir/data/spoken_language_identification/test_wav.sashdat')

In [12]:
test_feature_table = test_audio.create_audio_feature_table(label_level=-2, n_output_frames=1000)

In [13]:
test_feature_table.label_freq

,Level,Frequency
de,1,180
en,2,180
es,3,180


# Build the CNN+RNN Model

In [14]:
# build cnn + rnn models

model_cnnrnn = Sequential(conn=conn, model_table='cnnrnn')

init = 'msra'

# w40 * h1000
model_cnnrnn.add(InputLayer(name='input', n_channels=1, width = 40, height=1000))

# a reshape layer is required to convert the sequence input into a fixed-size tensor for CNN layers
model_cnnrnn.add(Reshape(width=40, height=1000, depth=1))

model_cnnrnn.add(Conv2D(n_filters=16, width=3, height=3, stride =1, init=init, act='identity', include_bias=False))
model_cnnrnn.add(BN())
model_cnnrnn.add(Pooling(width=3, height=3, stride =2))

# w20*h500              
model_cnnrnn.add(Conv2D(n_filters=32, width=3, height=3, stride =1, init=init, act='identity', include_bias=False))
model_cnnrnn.add(BN())
model_cnnrnn.add(Pooling(width=3, height=3, stride =2))

# w10*h250              
model_cnnrnn.add(Conv2D(n_filters=64, width=3, height=3, stride =1,init=init, act='identity', include_bias=False))
model_cnnrnn.add(BN())
model_cnnrnn.add(Pooling(width=3, height=3, stride =2))

# w5*h125*d64
# reshape the fixed-size tensor into a sequence data with 125 tokens, and token size is 5*64
model_cnnrnn.add(Reshape(order='DWH', width=5*64, height=125, depth=1))

model_cnnrnn.add(Recurrent(n=50, init='msra', rnn_type='lstm', output_type='samelength', dropout=0.8))
model_cnnrnn.add(Recurrent(n=50, init='msra', rnn_type='lstm', output_type='encoding', dropout=0.8))
              
#output              
model_cnnrnn.add(OutputLayer(n=3, name='output'))

NOTE: Input layer added.
NOTE: Reshape layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Pooling layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Pooling layer added.
NOTE: Convolution layer added.
NOTE: Batch normalization layer added.
NOTE: Pooling layer added.
NOTE: Reshape layer added.
NOTE: Recurrent layer added.
NOTE: Recurrent layer added.
NOTE: Output layer added.
NOTE: Model compiled successfully.


In [15]:
#%debug

In [ ]:
#model_cnnrnn.plot_network()

In [17]:
myvars = train_feature_table.feature_vars

In [18]:
# number of feature columns: 1000 (tokens)* 40 (token size)
len(myvars)

40000

In [19]:
#myvars

# Train the Model

In [20]:
from dlpy.lr_scheduler import *
from dlpy.model import *
lr_scheduler = ReduceLROnPlateau(conn=conn, cool_down_iters=5, gamma=0.1, learning_rate=0.01, patience=3)
solver = MomentumSolver(lr_scheduler = lr_scheduler,
                        clip_grad_max = 100, clip_grad_min = -100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=2, log_level=2, max_epochs=60, reg_l2=0.0005)

The following argument(s) learning_rate, gamma, step_size, power are overwritten by the according arguments specified in lr_scheduler.


In [21]:
data_specs_input = DataSpec(layer='input', type_='numnom', data=train_feature_table.feature_vars, 
                            numeric_nominal_parms=DataSpecNumNomOpts(length=train_feature_table.num_of_frames_col,
                                                                    token_size=train_feature_table.feature_size))

data_specs_output = DataSpec(layer='Output', type_='numnom', data='_label_')

In [22]:
model_cnnrnn.fit(data=train_feature_table,
              data_specs=[data_specs_input, data_specs_output],
              n_threads=16,
              seed=12598,
              record_seed=13544,
              gpu=dict(devices=2),
              train_from_scratch=True,
              force_equal_padding=True,
              optimizer=optimizer
             )

NOTE: Training from scratch.
NOTE: Using dlgrd009.unx.sas.com: 1 out of 4 available GPU devices.
NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 118361.
NOTE:  The approximate memory cost is 171.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost       7.62 (s).
NOTE:  The total number of threads on each worker is 16.
NOTE:  The total mini-batch size per thread on each worker is 2.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 32.
NOTE:  Target variable: _label_
NOTE:  Number of levels for the target variable:      3
NOTE:  Levels for the target variable:
NOTE:  Level      0: de
NOTE:  Level      1: en
NOTE:  Level      2: es
NOTE:  Number of input variables: 40000
NOTE:  Number of numeric input variables:  40000
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  0          0.01           1.105     0.6569    49.76
NOTE:  1          0.01           1.089       0.62    49.30

,Descr,Value
0,Model Name,cnnrnn
1,Model Type,Recurrent Neural Network
2,Number of Layers,15
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,3
6,Number of Pooling Layers,3
7,Number of Fully Connected Layers,0
8,Number of Recurrent Layers,2
9,Number of Batch Normalization Layers,3


# Assess the Model

In [23]:
model_cnnrnn.evaluate(data=test_feature_table, gpu=dict(devices=2), model_task='CLASSIFICATION')

NOTE: Due to data distribution, miniBatchSize has been limited to 9.
NOTE: Using dlgrd009.unx.sas.com: 1 out of 4 available GPU devices.


[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read           540
 1  Number of Observations Used           540
 2  Misclassification Error (%)      2.592593
 3                   Loss Error      0.072839

[OutputCasTables]

             casLib              Name  Rows  Columns  \
 0  CASUSER(username)  Valid_Res_0rCOAU   540    40010   
 
                                                  casTable  
 0  CASTable('Valid_Res_0rCOAU', caslib='CASUSER(username)')  

+ Elapsed: 6.11s, user: 27.9s, sys: 23.2s, mem: 2.32e+03mb

In [24]:
model_cnnrnn.valid_conf_mat

,_label_,Col1,Col2,Col3
0,de,172.0,3.0,5.0
1,en,3.0,176.0,1.0
2,es,2.0,0.0,178.0


In [25]:
test_result_table = model_cnnrnn.valid_res_tbl

In [26]:
# Show the confusion matrix
display(confusion_matrix(test_result_table['_label_'], test_result_table['I__label_']))

,de,en,es
_label_,,,
de,172.0,3.0,5.0
en,3.0,176.0,1.0
es,2.0,0.0,178.0


In [27]:
# Calculate the average f1 score
f1 = 0
for i in 0, 1, 2:
    f1 = f1 + f1_score(test_result_table['_label_'], test_result_table['I__label_'], pos_label=i)
print('the f1 score is {:.6f}'.format(f1/3))

the f1 score is 0.974036


In [28]:
# Calculate the accuracy score
acc_score = accuracy_score(test_result_table['_label_'], test_result_table['I__label_'])
print('the accuracy score is {:.6f}'.format(acc_score))

the accuracy score is 0.974074


# Deploy the Model

In [29]:
model_cnnrnn.deploy(output_format='table', 
                      path='/data/spoken_language_identification')

NOTE: Model table saved successfully.


In [32]:
model_cnnrnn.deploy(output_format='astore', 
                      path=r'\data\DeepLearn\data\spoken_language_identification')

NOTE: Model astore file saved successfully.


In [33]:
conn.endsession()

+ Elapsed: 9.1e-05s, sys: 8.3e-05s, mem: 0.203mb

In [34]:
#conn.shutdown()